## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-14-01-14-11 +0000,nypost,"NYC wannabe rapper accused of torturing, killi...",https://nypost.com/2026/01/13/us-news/nyc-wann...
1,2026-01-14-01-11-08 +0000,bbc,Chris Mason: The climbdowns stack up for Starmer,https://www.bbc.com/news/articles/c14regjvk6no...
2,2026-01-14-01-07-00 +0000,wsj,Oil Futures Edge Lower on Likely Technical Cor...,https://www.wsj.com/finance/commodities-future...
3,2026-01-14-01-06-58 +0000,nyt,"Claudette Colvin, Who Refused to Give Her Bus ...",https://www.nytimes.com/2026/01/13/us/politics...
4,2026-01-14-01-06-55 +0000,nypost,Ex-Navy Seal plotted to blast cops with ‘morta...,https://nypost.com/2026/01/13/us-news/ex-navy-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2385/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
64,trump,59
201,iran,34
144,ice,20
160,new,18
279,fed,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
134,2026-01-13-21-07-00 +0000,wsj,President Trump said the economy was poised fo...,https://www.wsj.com/politics/policy/trump-pred...,141
270,2026-01-13-14-47-00 +0000,wsj,Some Trump administration officials and allies...,https://www.wsj.com/politics/policy/trump-alli...,127
386,2026-01-13-02-38-00 +0000,wsj,White House Weighs Iran’s Nuclear-Talks Offer ...,https://www.wsj.com/world/middle-east/trump-ir...,122
273,2026-01-13-14-35-20 +0000,nyt,Trump Says He Will Impose Tariffs on Iran’s Tr...,https://www.nytimes.com/2026/01/13/world/middl...,121
83,2026-01-13-22-43-33 +0000,nypost,The potential targets Trump could strike in Ir...,https://nypost.com/2026/01/13/world-news/the-p...,118


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
134,141,2026-01-13-21-07-00 +0000,wsj,President Trump said the economy was poised fo...,https://www.wsj.com/politics/policy/trump-pred...
264,79,2026-01-13-15-06-00 +0000,wsj,The Justice Department’s investigation of Fede...,https://www.wsj.com/economy/central-banking/th...
393,70,2026-01-13-01-57-00 +0000,wsj,New York Fed President John Williams said that...,https://www.wsj.com/economy/central-banking/fe...
336,68,2026-01-13-10-14-18 +0000,nyt,Death Toll Grows as Nationwide Protests Rock I...,https://www.nytimes.com/2026/01/10/world/middl...
140,55,2026-01-13-20-51-30 +0000,cbc,"As Carney heads to China to talk trade, Ontari...",https://www.cbc.ca/news/canada/toronto/doug-fo...
71,54,2026-01-13-23-02-49 +0000,cbc,Federal prosecutors resign amid turmoil over M...,https://www.cbc.ca/news/world/federal-prosecut...
179,53,2026-01-13-18-51-55 +0000,latimes,Supreme Court likely to uphold state bans on t...,https://www.latimes.com/politics/story/2026-01...
89,49,2026-01-13-22-37-28 +0000,nypost,Eric Adams tells heckler ‘go f–k yourself’ as ...,https://nypost.com/2026/01/13/us-news/eric-ada...
120,43,2026-01-13-21-35-00 +0000,wsj,The Pentagon has told lawmakers that it chose ...,https://www.wsj.com/politics/national-security...
138,41,2026-01-13-20-53-36 +0000,nypost,Greenland’s prime minister rebuffs Trump’s eff...,https://nypost.com/2026/01/13/us-news/greenlan...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
